In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **reading data**

In [ ]:
train_features=pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
test_features=pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_targets_scored=pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_non_scored=pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
train_drug=pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')


In [ ]:
train_drug

In [ ]:
train_features

In [ ]:
train_features.shape

In [ ]:
#train_features.merge(train_targets_s,on='sig_id',
merged1=pd.merge(train_features,train_drug,how='outer',on='sig_id')
print(merged1.shape)
merged1

In [ ]:
#merged1.drop('sig_id',axis=1,inplace=True)

In [ ]:
merged1.drop('drug_id',axis=1,inplace=True)

# **import pytorch**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
merged1

In [ ]:
# x=merged1['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
# x

In [ ]:
#merged1.loc[merged1['cp_type']=='trt_cp','cp_type']
#merged1.loc[:, 'cp_type']

# **pre processing function**

In [ ]:
def preprocess(df):
    df['cp_type']=df['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
    df['cp_dose']=df['cp_dose'].map({'D1':0,'D2':1})
    df['cp_time']=df['cp_time'].map({24:0,48:1,72:2})
    
    return df   

In [ ]:
merged1=preprocess(merged1)
merged1

In [ ]:
test_sigid=test_features['sig_id']
test_features.drop('sig_id',axis=1,inplace=True)
test_features=preprocess(test_features)
test_features


In [ ]:
#!pip install iterative-stratification
#https://github.com/trent-b/iterative-stratification

In [ ]:
# import sys
# sys.path.append('../input/iterative-stratification/iterative-stratification-master')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
# train_scored_sig_id=train_targets_scored['sig_id']
# #train_targets_scored.drop('sig_id',axis=1,inplace=True)
# train_targets_scored

In [ ]:
merged1.drop('sig_id',axis=1,inplace=True)

In [ ]:
merged1

In [ ]:
train_targets_scored

In [ ]:
# Fold = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=42)
# x,y=Fold.split(merged1,train_targets_scored)
# x


In [ ]:
y

In [ ]:
#folds=merged1.copy()
# for n, (train_index, val_index) in enumerate(Fold.split(merged1, train_targets_scored)):
#     merged1.loc[val_index, 'fold'] = int(n)
#     train_targets_scored.loc[val_index,'fold']=int(n)
    
# merged1['fold'] = merged1['fold'].astype(int)
# train_targets_scored['fold']=train_targets_scored['fold'].astype(int)
# #print(folds.shape)
# train_targets_scored

In [ ]:
# y.len()

# x=torch.Tensor(x)
# y=torch.Tensor(y)


In [ ]:
# x=torch.Tensor(x)
# y=torch.Tensor(y)

In [ ]:
# x_train=merged1.loc[merged1['fold']==0].drop('fold',axis=1).to_numpy()
# x_test=merged1.loc[merged1['fold']==1].drop('fold',axis=1).to_numpy()
# #x_test

In [ ]:
#x_test

In [ ]:
# y_train=train_targets_scored.loc[train_targets_scored['fold']==0].drop((['fold','sig_id']),axis=1).to_numpy()
# y_test=train_targets_scored.loc[train_targets_scored['fold']==0].drop((['fold','sig_id']),axis=1).to_numpy()
# #y_test

In [ ]:
#y_test.size

In [ ]:
# x_train_tensors=torch.Tensor(x_train)
# x_test_tensors=torch.Tensor(x_test)
# y_train_tensors=torch.Tensor(y_train)
# y_test_tensors=torch.Tensor(y_test)
# #print(train_data_dummy)

In [ ]:
# for i in range(100,200):
#     if((2452842%i)==0):
#         print(i)

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [ ]:
# train_ds = TensorDataset(x_train_tensors,y_train_tensors)
#train_ds[:2]          

In [ ]:
# batch_size=126
# train_dl=DataLoader(train_ds, batch_size=batch_size)

In [ ]:
#train_ds.size

In [ ]:
# for x,y in train_dl:
#     print(x.shape,y.shape)

# **Neural net**

In [ ]:
class MOA(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(875,200)
        #self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150,100)
        self.fc4 = nn.Linear(100,207)
    def forward(self, x):
        x = self.fc1(x)
        #x = self.dropout1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.sigmoid(x)

In [ ]:
model=MOA()

In [ ]:
targets=[col for col in train_targets_scored.columns]

In [ ]:
# def mean_log_loss(y_true, y_pred):
#     metrics = []
#     for i, target in enumerate(targets):
#         metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
#     return np.mean(metrics)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# count=0
# loss=0
# def fit(num_epochs, model, loss_fn, opt):
#     print("loss")
#     #loss = AverageMeter()
#     for epoch in range(num_epochs):
#         for xb,yb in train_dl:
            
#             # Generate predictions
#             global count;
#             opt.zero_grad()
#             pred = model(xb)
#             #loss = loss_f(pred, yb)
#             #print(yb)
#             loss = loss_fn(pred, yb)
#             #loss = nn.BCEWithLogitsLoss()(pred, yb)
# #             loss.update(loss.item(), batch_size)
# #             if CFG.gradient_accumulation_steps > 1:
# #                 loss = loss / CFG.gradient_accumulation_steps
#             # Perform gradient descent
#             loss.backward()
            
#             opt.step()
            
#             #print(pred)
#             count+=1
#             if(count%10==0):
#                 print("lossed")
#                 print(count,loss)
#     #print('Training loss: ', loss)

In [ ]:
#fit(3,model,loss_fn,opt)

In [ ]:
test_features

In [ ]:
test_features_nump=test_features.to_numpy()

In [ ]:
x_test_tensors=torch.Tensor(test_features_nump)

In [ ]:
pred=model(x_test_tensors)

In [ ]:
pred

In [ ]:
pred_nump=pred.detach().numpy()

In [ ]:
pred_df=pd.DataFrame(pred_nump)

In [ ]:
pred_df.columns=col

In [ ]:
pred_df['sig_id']=test_sigid

In [ ]:
pred_df.columns[0]=pred_df['sig_id']


In [ ]:
pred_df

In [ ]:
pred_df.to_csv('submission.csv', index=False)

In [ ]:
col=train_targets_scored.columns.difference(['sig_id'])

In [ ]:
#train_targets_scored